In [1]:
pip install git+https://github.com/Desklop/Uk_Stemmer

  Cloning https://github.com/Desklop/Uk_Stemmer to /tmp/pip-req-build-3rnump56
  Running command git clone -q https://github.com/Desklop/Uk_Stemmer /tmp/pip-req-build-3rnump56


In [2]:
pip install spark-nlp==3.4.0

  Using cached spark_nlp-3.4.0-py2.py3-none-any.whl (140 kB)
  Attempting uninstall: spark-nlp
    Found existing installation: spark-nlp 3.3.4
    Uninstalling spark-nlp-3.3.4:
      Successfully uninstalled spark-nlp-3.3.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nlu 3.3.1 requires spark-nlp<3.4.0,>=3.3.0, but you have spark-nlp 3.4.0 which is incompatible.


In [3]:
pip install pyspark==3.0.2

In [4]:
pip install nlu pyspark==3.0.2

  Using cached spark_nlp-3.3.4-py2.py3-none-any.whl (133 kB)
  Attempting uninstall: spark-nlp
    Found existing installation: spark-nlp 3.4.0
    Uninstalling spark-nlp-3.4.0:
      Successfully uninstalled spark-nlp-3.4.0


In [5]:
!pip install pyenchant

In [6]:
!apt-get install libenchant1c2a

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libenchant1c2a is already the newest version (1.6.0-11.1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from uk_stemmer import UkStemmer

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp

In [10]:
import nlu

In [11]:
import enchant

In [12]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
pd.options.display.float_format = "{:,.2f}".format
np.set_printoptions(suppress=True)
nltk.download('stopwords')
random_state = 8080

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
df = pd.read_csv("full_data.csv")

In [16]:
df.head()

,crm_customer_id,created,NAME_LOWEST
0,50081964,2021-01-25 14:21:07.610,Петлі меблеві
1,50081964,2021-01-25 14:21:07.610,Петлі меблеві
2,50081964,2021-01-29 13:53:52.920,Лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,Інсектициди
4,50081964,2021-02-20 15:02:06.730,Рейлінги для кухні


In [17]:
df.columns = ["customer_id", "date", "category"]

In [18]:
df.head(2)

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,Петлі меблеві
1,50081964,2021-01-25 14:21:07.610,Петлі меблеві


In [19]:
df.shape

(854919, 3)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 854919 entries, 0 to 854918
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  854919 non-null  int64 
 1   date         854919 non-null  object
 2   category     854918 non-null  object
dtypes: int64(1), object(2)
memory usage: 19.6+ MB


In [21]:
print(f"number of categories: {df.category.nunique()}")

number of categories: 2065


In [22]:
print(f"number of customers: {df.customer_id.nunique()}")

number of customers: 10669


# NLP feature engineering

1. make all words lowercase
2. remove stopwords
3. preform stemming

In [23]:
df.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,Петлі меблеві
1,50081964,2021-01-25 14:21:07.610,Петлі меблеві
2,50081964,2021-01-29 13:53:52.920,Лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,Інсектициди
4,50081964,2021-02-20 15:02:06.730,Рейлінги для кухні


In [24]:
df.category.unique()[:20]

array(['Петлі меблеві', 'Лотки для столових приборів', 'Інсектициди',
       'Рейлінги для кухні', 'Алюмінієвий профіль', 'Стопори дверні',
       'Зубна паста', 'Куточок алюмінієвий', 'Мішки для сміття',
       'Господарчі рукавички', 'Серветки для прибирання',
       'Аератори для крана', 'Засоби для ванних кімнат', 'Швабри',
       'Засоби для кухні', 'Засоби для догляду за унітазом',
       'Клей побутовий', 'Губки господарчі', 'Пральний порошок',
       'Клей для шпалер'], dtype=object)

In [25]:
"для" in stopwords.words("ukrainian")

True

In [26]:
ndf = df.drop_duplicates(subset=["customer_id", "category"])

In [27]:
print(f"distinct records: {round((ndf.shape[0] * 100) / df.shape[0])}%")

distinct records: 37%


In [28]:
ndf.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,Петлі меблеві
2,50081964,2021-01-29 13:53:52.920,Лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,Інсектициди
4,50081964,2021-02-20 15:02:06.730,Рейлінги для кухні
6,50081964,2021-02-20 15:02:06.730,Алюмінієвий профіль


In [29]:
ndf["category"] = ndf["category"].str.lower()

In [30]:
ndf.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,петлі меблеві
2,50081964,2021-01-29 13:53:52.920,лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,інсектициди
4,50081964,2021-02-20 15:02:06.730,рейлінги для кухні
6,50081964,2021-02-20 15:02:06.730,алюмінієвий профіль


In [31]:
sum(ndf.category.str.isnumeric())

nan

In [32]:
categories = ndf.category.unique()

In [33]:
categories

array(['петлі меблеві', 'лотки для столових приборів', 'інсектициди', ...,
       'підсвітки автомобільні', 'бігові доріжки', nan], dtype=object)

In [34]:
ndf.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,петлі меблеві
2,50081964,2021-01-29 13:53:52.920,лотки для столових приборів
3,50081964,2021-05-31 12:21:22.080,інсектициди
4,50081964,2021-02-20 15:02:06.730,рейлінги для кухні
6,50081964,2021-02-20 15:02:06.730,алюмінієвий профіль


In [35]:
without_stopwords_lemma_df = nlu.load('uk.lemma').predict(ndf.head().category, output_level='document')
without_stopwords_lemma_df.lem

lemma download started this may take some time.
Approximate size to download 2.3 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


0                    [петля, меблеві]
1    [лотки, для, столовий, приборів]
2                       [інсектициди]
3              [рейлінги, для, кухня]
4              [алюмінієвий, профіль]
Name: lem, dtype: object

In [36]:
stem_df = nlu.load('uk.stem').predict(ndf.head().category, output_level='document')
stem_df.stem

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


0                    [петлі, меблеві]
1    [лотки, для, столових, приборів]
2                       [інсектициди]
3              [рейлінги, для, кухні]
4              [алюмінієвий, профіль]
Name: stem, dtype: object

In [41]:
categories

array(['петлі меблеві', 'лотки для столових приборів', 'інсектициди', ...,
       'підсвітки автомобільні', 'бігові доріжки', nan], dtype=object)

In [42]:
for category in categories[:-1]:
    processed_category = [word for word in word_tokenize(category)
                              if (word not in stopwords.words("ukrainian")) and word.isalpha()]
    ndf.loc[ndf["category"] == category, "category"] = ' '.join(processed_category)

In [43]:
ndf.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,петлі меблеві
2,50081964,2021-01-29 13:53:52.920,лотки столових приборів
3,50081964,2021-05-31 12:21:22.080,інсектициди
4,50081964,2021-02-20 15:02:06.730,рейлінги кухні
6,50081964,2021-02-20 15:02:06.730,алюмінієвий профіль


In [44]:
categories = ndf.category.unique()

In [45]:
categories = pd.Series(categories)
categories

0                 петлі меблеві
1       лотки столових приборів
2                   інсектициди
3                рейлінги кухні
4           алюмінієвий профіль
                 ...           
2022    аксесуари ванних кімнат
2023          електровелосипеди
2024     підсвітки автомобільні
2025             бігові доріжки
2026                        NaN
Length: 2027, dtype: object

In [46]:
categories.shape

(2027,)

In [48]:
lemma_categories = nlu.load('uk.lemma').predict(categories[:-1], output_level='document')

lemma download started this may take some time.
Approximate size to download 2.3 MB
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [49]:
lemma_categories.rename({"document": "category"}, axis=1, inplace=True)

In [50]:
lemma_categories.head()

,category,lem
0,петлі меблеві,"[петля, меблеві]"
1,лотки столових приборів,"[лотки, столовий, приборів]"
2,інсектициди,[інсектициди]
3,рейлінги кухні,"[рейлінги, кухня]"
4,алюмінієвий профіль,"[алюмінієвий, профіль]"


In [51]:
ndf.head()

,customer_id,date,category
0,50081964,2021-01-25 14:21:07.610,петлі меблеві
2,50081964,2021-01-29 13:53:52.920,лотки столових приборів
3,50081964,2021-05-31 12:21:22.080,інсектициди
4,50081964,2021-02-20 15:02:06.730,рейлінги кухні
6,50081964,2021-02-20 15:02:06.730,алюмінієвий профіль


In [52]:
ndf = ndf.merge(lemma_categories, on="category")

In [53]:
ndf.head()

,customer_id,date,category,lem
0,50081964,2021-01-25 14:21:07.610,петлі меблеві,"[петля, меблеві]"
1,50171691,2021-02-17 13:51:00.350,петлі меблеві,"[петля, меблеві]"
2,1600035308,2021-05-09 14:28:16.517,петлі меблеві,"[петля, меблеві]"
3,2000001033,2020-10-17 14:38:24.470,петлі меблеві,"[петля, меблеві]"
4,2000001042,2021-04-20 19:50:38.070,петлі меблеві,"[петля, меблеві]"


In [54]:
ndf

,customer_id,date,category,lem
0,50081964,2021-01-25 14:21:07.610,петлі меблеві,"[петля, меблеві]"
1,50171691,2021-02-17 13:51:00.350,петлі меблеві,"[петля, меблеві]"
2,1600035308,2021-05-09 14:28:16.517,петлі меблеві,"[петля, меблеві]"
3,2000001033,2020-10-17 14:38:24.470,петлі меблеві,"[петля, меблеві]"
4,2000001042,2021-04-20 19:50:38.070,петлі меблеві,"[петля, меблеві]"
...,...,...,...,...
320531,19000079909,2021-04-25 11:54:35.393,аксесуари ванних кімнат,"[аксесуари, ванний, кімната]"
320532,19000080121,2021-05-25 18:51:02.880,електровелосипеди,[електровелосипеди]
320533,19000082934,2021-04-30 20:53:32.137,електровелосипеди,[електровелосипеди]
320534,19000081912,2020-10-18 18:11:54.680,підсвітки автомобільні,"[підсвітки, автомобільний]"


In [73]:
customers_queries = ndf.set_index(["customer_id", "date"]).drop("category", axis=1)

In [76]:
customers_queries.head()["lem"].tolist()

[['петля', 'меблеві'],
 ['петля', 'меблеві'],
 ['петля', 'меблеві'],
 ['петля', 'меблеві'],
 ['петля', 'меблеві']]

In [81]:
# customers_queries = pd.DataFrame(customers_queries.lem.tolist())


In [56]:
customers_queries["lem"] = customers_queries["lem"].astype(str)

In [72]:
customers_queries.lem.str.split(",\s*", expand=True).iloc[:].stack().reset_index().pivot_table(index="customer_id", columns=0, fill_value=0, aggfunc='size')

,'apple'],'bis'],'honor'],'horeca'],'led'],'lego'],'samsung'],'аварійний','аварійний'],'авто'],'автодиагностики'],'автокрісел'],'автомасажери'],'автомат'],'автомобіль'],'автомобільний'],'автотехнікс'],'адаптери','адміністративний'],'акб'],'аксесуари','аксесуари'],'аксесуарів'],'активний','акумуляторні'],'алмазного','алмазні'],'альпіністів'],'алюмінієвий'],'анкери'],'антена'],'антикорозійної','антипробуксовувальні','апарат'],'аплікація'],'аптечки'],'арбалета','арматура'],'аромадифузорів'],'аромаламп',...,['штативи',['штемпельна',['штопори',['штора'],['шторки',['штроборізи'],['штукатурка'],['штучний',['шумовіброізоляція',['шумівки'],['шурупи'],['шуруповерти'],['щебінь'],['щит',['щітки',['яйцерізки'],['якір',['ящик',['ємність',['іграшка',['іграшка'],['іграшкові',['ігровий',['ізоляційні',['ізоляція',['інвентар',['інвертори',['індексні',['індикатор',['інсектициди'],['інсталяції'],['інструмент',['інструмент'],['інструментальний',['інтерактивні',['інфрачервоні',['ґрунт',['ґрунтовка',['ґрунтовка'],nan
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50081964,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
50095759,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
50097247,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
50113709,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
50116694,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19000084342,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19000084350,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
19000084360,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
